# Pu-238 Maker

In this notebook I will experiment with the OpenMC tools needed to estimate Pu-238 production rate due to irradiation of a NpO2/Al cermet target.  

In [1]:
import openmc
import openmc.deplete # will be needed for depletion analysis eventually

%matplotlib inline
import numpy as np

## Materials

While I would expect the driver fuel pins (or other driver fuel forms) will be of relatively easy to define composition, I do not have a firm grasp on the composition of the NpO2/Al CERMET target. What follows is my best guess based on information provided in: 

Howard, Richard H. Overview of the Plutonium-238 Supply Program's CERMET Production Target. Oak Ridge National Lab.(ORNL), Oak Ridge, TN (United States), 2019.

In [6]:
NpO2 = openmc.Material();
NpO2.add_nuclide('Np237',1);
NpO2.add_element('O',2);
NpO2.set_density('g/cc',11.143);

Al = openmc.Material();
Al.add_element('Al',1);
Al.set_density('g/cc',2.7);

target = openmc.Material.mix_materials([NpO2,Al],[0.2,0.7],'vo'); # note: 10% is void.


In [7]:
print(target.get_nuclide_atom_densities());

OrderedDict([('Np237', ('Np237', 0.004988479277370189)), ('O16', ('O16', 0.009973177287448133)), ('O17', ('O17', 3.781267292246604e-06)), ('Al27', ('Al27', 0.042183829047439676))])


In [8]:
print(target.get_mass_density())

4.1186


I think this is at least reasonable.  